In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [5]:
df = pd.read_csv('../data/train_data_encoded.csv')
print('shape:', df.shape)
df.head(5)

shape: (36944, 8)


,quality,color,clarity,price,weight_ES,depth_percent_ES,table_percent_ES,volume_ES
0,3,6,1,6.353,-0.625000,0.357143,0.333333,-0.591767
1,4,5,5,9.183,0.484375,0.571429,-0.333333,0.550637
2,4,4,3,7.983,0.031250,-0.071429,0.666667,0.052269
3,2,3,1,8.371,0.593750,0.928571,0.000000,0.624043
4,3,3,4,6.588,-0.531250,0.285714,0.666667,-0.511469


In [6]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print('\n')
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    29555.000000
mean         7.702645
std          0.976815
min          5.814000
25%          6.809000
50%          7.689000
75%          8.505000
max          9.842000
Name: price, dtype: float64


Datos de testeo
-----------------------
count    7389.000000
mean        7.699702
std         0.982903
min         5.787000
25%         6.802000
50%         7.695000
75%         8.511000
max         9.841000
Name: price, dtype: float64


In [7]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [8]:
param = {"max_depth": [18],
        "max_features": [15],
        "min_samples_split": [10]}

random_forest = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param,
            cv=10,
            n_jobs = -1,
            verbose= 3,
            return_train_score = True,
            scoring="neg_mean_squared_error")

random_forest.fit(X_train, y_train)

best_rf = random_forest.best_estimator_
print(best_rf)

y_pred_test_rf = random_forest.predict(X_test)
y_pred_train_rf = random_forest.predict(X_train)
rf_results_1 = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest 1")
rf_results_1.style.background_gradient(cmap='coolwarm')

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 7/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.011) total time=  23.5s
[CV 5/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.010) total time=  24.2s
[CV 8/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.011) total time=  24.3s
[CV 2/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.010) total time=  24.2s
[CV 3/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.011) total time=  24.4s
[CV 6/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.011) total time=  24.7s
[CV 1/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, test=-0.010) total time=  25.2s
[CV 4/10] END max_depth=18, max_features=15, min_samples_split=10;, score=(train=-0.005, tes

,MAE,MSE,RMSE,R2,set,modelo
0,0.077483,0.010442,0.102188,0.989190,test,Random Forest 1
1,0.051924,0.004716,0.068672,0.995057,train,Random Forest 1


In [9]:
df_test = pd.read_csv('../data/test_data_encoded.csv')
df_test

,quality,color,clarity,weight_ES,depth_percent_ES,table_percent_ES,volume_ES
0,4,1,2,-0.609375,-1.000000,0.333333,-0.586243
1,3,1,2,0.828125,0.714286,1.000000,0.758368
2,3,6,2,1.484375,0.071429,0.666667,1.465556
3,3,6,1,0.062500,-0.928571,-0.333333,0.098797
4,0,5,1,1.234375,2.071429,-0.666667,1.206485
...,...,...,...,...,...,...,...
13480,3,3,2,0.609375,-1.642857,1.000000,0.619552
13481,2,6,2,0.296875,0.142857,1.000000,0.288656
13482,4,4,3,-0.640625,0.142857,-1.233333,-0.623494
13483,4,1,2,0.843750,-1.642857,0.666667,0.886611


In [10]:
#predict test to submit competition
y_pred_gb_test = random_forest.predict(df_test)
submission_2 = pd.DataFrame(y_pred_gb_test).reset_index()
print('shape:', y_pred_gb_test.shape)
submission_2.columns = ['id', 'price']
submission_2

shape: (13485,)


,id,price
0,0,6.118563
1,1,8.540708
2,2,9.446798
3,3,7.851999
4,4,8.957871
...,...,...
13480,13480,8.490797
13481,13481,8.318207
13482,13482,6.299688
13483,13483,8.635579


In [12]:
submission_2.to_csv('../output/submission_2.csv', index=False)